##### Copyright 2018 The TF-Agents Authors.

# 简介

强化学习是一个通用的框架，在这个框架中可以让代理（agent）在对应的环境（environment）中执行相应的动作，以达到最大化回报（reward）的目的。其中涉及到的两个主要内容，其中一个为环境（environment），它代表了一个需要解决的问题；另一个为代理（agent），它代表了一种学习的算法。（Agent可以表示一个采用特定算法进行学习的个体）

代理和环境之间不断的相互作用。在每一次的迭代中，每一个代理根据自己的策略（policy）$\pi(a_t|s_t)$采取相应的动作（action），其中$s_t$是当前从环境中的观察，获得的回报为 $r_{t+1}$，下一次从环境中的观察为$s_{t+1}$。目标是提升策略以至于最大化回报（并返回）。

注意：区分环境中的`状态`（`state`）和`观察`（`observation`）很重要，代理可以从环境中看见的称之为`状态`，例如在扑克游戏中，环境状态由属于所有玩家的纸牌和整体的纸牌组成，但个体只能看到他自己的纸牌和整体纸牌的一部分。在大多数文献中，这些术语可互换使用，观察值也表示为$s$

![Agent-Environment Interation Loop](images/rl_overview.png)
这是一个非常通用的框架，可以模拟各种顺序决策问题，例如游戏、机器人等。

# Cartpole环境

Cartpole环境是最经典的强化学习问题之一（类似于强化学习中的*"Hello, World!"* ）。杆子连接到手推车，手推车可以沿着无摩擦的轨道移动。 杆子开始直立，目的是通过控制手推车防止其跌落。


- 从环境中观察得到的$s_t$是一个4维的向量，代表了手推车的位置和速度，以及杆子的角度和角速度。
- 代理可以通过两个动作$a_t$来控制这个系统：将手推车推向右边（+1）或左边（-1）。
- 杆保持直立的每个时间步长都会提供奖励$r_ {t + 1} = 1$。 如果满足以下任一条件，则情节结束：
    - 杆子超过某个角度的限制
    - 手推车移出了设置的边界
    - 经过了200时间步长
   
代理的目标是学习一个策略$\pi(a_t|s_t)$以至于再一个时间段内可以最大化回报的总和$\sum_{t=0}^{T} \gamma^t r_t$。其中$\gamma$是一个在$[0, 1]$之间的衰减因子，相当于未来的奖励是对当前的奖励的折扣。此参数有助于我们集中精力制定策略，使其更加关注快速获得奖励的情况。

# DQN 代理

DQN  [DQN (Deep Q-Network) algorithm](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)算法在2015年由DeepMind团队发明。通过结合大规模的强化学习和深度神经网络，它能够解决各种Atari游戏（一些还可一达到超越人类的水平）。此算法使用结合深度神经网络的Q-learning和“经验重播”（experience replay）技来增强传统强化学习。

## Q-Learning

Q-Learning是基于Q函数的概念，一个策略$\pi$的Q函数（又称为状态作用值函数）为$Q^{\pi}(s, a)$，通过首先采取行动$a$，然后采取策略$\pi$来衡量从状态$s$获得的预期收益或奖励的折扣总和。我们定义一个优化的Q函数$Q^*(s, a)$作为最大化的返回值，它可以从观察值$s$开始，采取行动$a$，然后遵循最佳策略来获得。 最佳Q函数遵循以下*Bellman*最佳性方程：

$\begin{equation}
Q^*(s, a) = \mathbb{E}\left[ r + \gamma \max_{a'} Q^*(s', a')\right]
\end{equation}$

这表明从状态$s$和动作$a$得到的最大化返回值是由当前返回值$r$和遵循最优化策略之后直到一个阶段结束得到返回值（经过$\gamma$衰减）的总和组成（例如从下一个状态$s'$得到的最大化的回报）。期望值是在立即奖励$r$的分布以及可能的下一个状态$s'$的基础上计算的。

Q-Learning的基本思想是使用Bellman最优方程作为迭代更新$Q_{i+1}(s, a) \leftarrow \mathbb{E}\left[ r + \gamma \max_{a'} Q_{i}(s', a')\right]$，可以证明这收敛到最优的$Q$函数，例如$Q_i \rightarrow Q^*$作为$i \rightarrow \infty$。详情查看 ([DQN 论文](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)).


## Deep Q-Learning
对于大多数问题，将$Q$函数表示为包含$s$和$s$每种组合的值所构成的表是不切实际的。相反，我们训练一个函数逼近器（例如带有参数$\theta$的神经网络）来估算Q值，即$Q(s,a;\theta)\approx Q^*(s,a)$。 这可以通过使每个步骤$i$的以下损失最小化来完成：

$\begin{equation}L_i(\theta_i) = \mathbb{E}_{s, a, r, s'\sim \rho(.)} \left[ (y_i - Q(s, a; \theta_i))^2 \right]\end{equation}$ where $y_i = r +  \gamma \max_{a'} Q(s', a'; \theta_{i-1})$

其中，$y_i$被称为TD (temporal difference，时间差异)代理，$y_i - Q$被称为TD误差，$\rho$表示行为的分布，从环境中收集的过渡$\{s，a，r，s'\}$的分布。

请注意，来自先前迭代$\theta_{i-1}$的参数是固定的，不会更新。 实际上，我们使用前几次迭代的网络参数作为快照而不是最后一次迭代的， 这个副本称为“目标网络”（*target network*.）。

Q-Learning是一种*off-policy* 的算法，它是一个贪心策略$a = \max_{a} Q(s, a; \theta)$ ，同时对于环境或收集的数据使用不同的行为策略。这个行为策略往往是一个$\epsilon$-贪婪的策略，它以$1-\epsilon$的概率和$\epsilon$概率的随机行为来选择贪婪的动作，以此来保证广泛覆盖的状态动作空间。

## Experience Replay
为了避免计算DQN损失的全部期望，我们可以使用随机梯度下降法使其最小化。如果仅使用最后一个转换$\{s, a, r, s'\}$来计算损失，那么就可以简化为标准的Q-Learning。

Atari DQN的工作引入了一种称为经验重播的技术，使网络更新更加稳定。在数据收集的每个时间步骤中，转换被添加到一个称为*replay buffer*的循环缓冲区中。然后在训练期间，我们不再使用最新的转换来计算损耗及其梯度，而是使用从重播缓冲区中取样的一小批转换来计算损耗及其梯度。这样做有两个好处:在许多更新中重用每个转换，从而提高数据效率;在批处理中使用不相关的转换，从而提高稳定性。


# 使用TF-Agents中的DQN 处理 Cartpole问题
TF-Agents提供训练DQN代理所需的所有组件，例如代理本身、环境、策略、网络、重播缓冲区、数据收集循环和指标。这些组件是作为Python函数或TensorFlow图形操作来实现的，而且我们还有用于在它们之间进行转换的包装器。此外，TF- agents支持TensorFlow 2.0模式，这使我们能够在命令式模式下使用TF。

下一节教程： tutorial for training a DQN agent on the Cartpole environment using TF-Agents：使用使用TF-Agents中的DQN 处理Cartpole问题
